In [1]:
import numpy as np
from itertools import combinations, chain
from collections import Counter
from qiskit_algorithms.optimizers import POWELL

# 假设有 3 台机器和 3 个操作
n = 2  # 作业数
m = 3  # 机器数
l = 1  # C_max 的二进制位数

# 处理时间矩阵 (例如)
p_ij = np.array([[2, 3, 1], [4, 1, 2]])

# W = [3]  # 不可用机器索引 [1, 1]
# b = []  # 机器二进制表示
# for j in range(m):
#     binary_j = format(j, '0{}b'.format(N))
#     b_jk = [int(bit) for bit in binary_j]
#     b.append(b_jk)
# print(b)
# 常数
K1 = 100
P = 80
# 量子比特数量
num_qubits = n * m + m * l + l  # x_ij, tau_jh, z_h

In [2]:
# 2. XY_mixer
from qiskit.circuit.library.standard_gates import RYGate
from qiskit import QuantumCircuit, Aer, execute
from math import sqrt
from qiskit.visualization import plot_histogram
import math


def append_SCS_term(m, l, n):
    qc = QuantumCircuit(n, name='SCS_' + str(m) + ',' + str(l))
    m = m - 1
    for i in range(l):
        if (i + 1) == 1:
            qc.cx(m - 1, m)
            theta = sqrt(1 / (m + 1))
            c3ry_gate = RYGate(2 * math.acos(theta)).control(1)
            qc.append(c3ry_gate, [m, m - 1])
            qc.cx(m - 1, m)
        else:
            qc.cx(m - (i + 1), m)
            theta = sqrt((i + 1) / (m + 1))
            c3ry_gate = RYGate(2 * math.acos(theta)).control(2)
            qc.append(c3ry_gate, [m, m - (i + 1) + 1, m - (i + 1)])
            qc.cx(m - (i + 1), m)
    return qc


def get_Dicke_state_init_circuit(n, m, B, l):
    total_qubits = n * m + m * l + l
    qc = QuantumCircuit(total_qubits, name='modified_dicke_init')

    # 对前 n * m 个量子比特分组，每组 m 个量子比特中包含 B 个激发态
    for group in range(n):
        start = group * m  # 每组的起始索引
        for j in range(B):
            qc.x(start + j)  # 在每组的前 B 个比特上施加 X 门，使其为激发态

    # 在每组 m 个量子比特中应用 SCS 子电路生成激励共享态
    for group in range(n):
        start = group * m
        for i in range(m - 1):
            if m - i > B:
                qc.append(append_SCS_term(m - i, B, m), range(start, start + m))
            else:
                qc.append(append_SCS_term(m - i, m - i - 1, m), range(start, start + m))
            qc.barrier()

    # 额外的 l 个量子比特可用于纠缠初始化或其他操作
    for i in range(m * l + l):
        qc.h(n * m + i)

    return qc


In [3]:
# # n = 6 # 变量总数
# B = 1  # 限制1的个数
# # m = 2
# # k = 6
# qc = get_Dicke_state_init_circuit(n, m, B, l)
# dicke_circuit = QuantumCircuit(n * m + m * l + l, n * m + m * l + l)
# dicke_circuit.append(qc, range(n * m + m * l + l))
# dicke_circuit.measure(range(n * m + m * l + l), range(n * m + m * l + l))
# backend = Aer.get_backend('qasm_simulator')
# counts = execute(dicke_circuit, backend, shots = 10000).result().get_counts()
# print(counts)
# plot_histogram(counts)

In [4]:
# 多体 RZZ 门函数
def append_multi_rzz_term(qc, qubits, angle):
    if len(qubits) == 1:
        qc.rz(2 * angle, qubits[0])
    else:
        for i in range(len(qubits) - 1):
            qc.cx(qubits[i], qubits[i + 1])
        qc.rz(2 * angle, qubits[-1])
        for i in range(len(qubits) - 2, -1, -1):
            qc.cx(qubits[i], qubits[i + 1])

In [5]:
def append__zz_term(qc, qubit1, qubit2, gamma):
    qc.cx(qubit1, qubit2)
    qc.rz(2 * gamma, qubit2)
    qc.cx(qubit1, qubit2)


def append__z_term(qc, qubit, gamma):
    qc.rz(gamma, qubit)


def append__mixer_term(qc, qubit, beta):
    qc.rx(2 * beta, qubit)

\begin{align}\sum_{h=0}^{l-1} 2^h z_h\end{align}

In [6]:
def get_cost_circuit1(gamma, qc):
    # H_1: sum_h 2^h z_h
    for h in range(l):
        qc.rz(-gamma * 2 ** h / 2, n * m + m * l + h)  # 单体项 -2^h/2 Z_h

\begin{align}
H_2=K_1 n \frac{(m-2)^2+m}{4} I-K_1 \frac{m-2}{2} \sum_{i=1}^n \sum_{j=1}^m Z_{i j}+\frac{K_1}{4} \sum_{i=1}^n \sum_{j \neq j^{\prime}} Z_{i j} Z_{i j^{\prime}}
\end{align}

In [7]:
import itertools


def get_cost_circuit2(gamma, qc):
    # H_2: K1 * sum_i (sum_j x_ij - 1)^2
    for i in range(n):
        # 单体项
        for j in range(m):
            qubit = i * m + j
            qc.rz(-gamma * K1 * (m - 2) / 2, qubit)  # -K1 (m-2)/2 Z_ij
        # 双体项
        for j in range(m):
            for jp in range(j + 1, m):
                qubit1 = i * m + j
                qubit2 = i * m + jp
                append__zz_term(qc, qubit1, qubit2, gamma * K1 / 4)  # K1/4 Z_ij Z_ij'


\begin{align}K_2 \sum_{j=1}^m \left( \sum_{i=1}^n x_{ij} p_{ij} + \sum_{h=0}^{l-1} 2^h (\tau_{jh} - z_h) \right)^2\end{align}

In [8]:
# import itertools
from collections import Counter


def get_cost_circuit3(gamma, qc):
    # H_3: K2 * sum_j (sum_i x_ij p_ij + sum_h 2^h (tau_jh - z_h))^2
    for j in range(m):
        # 单体项系数
        const = (sum(p_ij[i][j] for i in range(n)) - sum(2 ** h for h in range(l))) / 2
        # sum_i p_ij Z_ij
        for i in range(n):
            qubit = i * m + j
            qc.rz(-gamma * P * const * p_ij[i][j] / 2, qubit)
        # sum_h 2^h Z_jh
        for h in range(l):
            qubit = n * m + j * l + h
            qc.rz(-gamma * P * const * 2 ** h / 2, qubit)
        # -sum_h 2^h Z_h
        for h in range(l):
            qubit = n * m + m * l + h
            qc.rz(gamma * P * const * 2 ** h / 2, qubit)  # 注意正负号

        # 双体项
        # Z_ij Z_i'j
        for i in range(n):
            for ip in range(i + 1, n):
                qubit1 = i * m + j
                qubit2 = ip * m + j
                append__zz_term(qc, qubit1, qubit2, gamma * P * p_ij[i][j] * p_ij[ip][j] / 4)
        # Z_jh Z_jh'
        for h in range(l):
            for hp in range(h + 1, l):
                qubit1 = n * m + j * l + h
                qubit2 = n * m + j * l + hp
                append__zz_term(qc, qubit1, qubit2, gamma * P * 2 ** (h + hp) / 4)
        # Z_h Z_h'
        for h in range(l):
            for hp in range(h + 1, l):
                qubit1 = n * m + m * l + h
                qubit2 = n * m + m * l + hp
                append__zz_term(qc, qubit1, qubit2, gamma * P * 2 ** (h + hp) / 4)
        # 交叉项（示例：Z_ij Z_jh, Z_ij Z_h）
        for i in range(n):
            for h in range(l):
                qc.rzz(gamma * P * p_ij[i][j] * 2 ** h / 4, i * m + j, n * m + j * l + h)  # Z_ij Z_jh
                qc.rzz(-gamma * P * p_ij[i][j] * 2 ** h / 4, i * m + j, n * m + m * l + h)  # -Z_ij Z_h
            for h in range(l):  # 修正部分
                for hp in range(l):  # 遍历所有 h, h' 对
                    if h != hp:  # 避免 h = h'
                        qubit1 = n * m + j * l + h
                        qubit2 = n * m + m * l + hp
                        qc.rzz(-gamma * P * 2 ** (h + hp) / 4, qubit1, qubit2)  # -Z_jh Z_h




In [9]:
def get_mixer_circuit(beta, qc):
    for i in range(num_qubits):
        qc.rx(2 * beta, i)
    return qc

In [10]:
def get_mixer_circuit_xy(beta, qc):
    for i in range(n):
        for j1 in range(m):
            for j2 in range(m):
                if j1 < j2:
                    qc.rxx(2 * beta, i * m + j1, i * m + j2)
                    qc.ryy(2 * beta, i * m + j1, i * m + j2)

In [11]:
# def invert_counts(s):
#     return s[::-1]

In [64]:
from qiskit import Aer, QuantumCircuit, execute


# 定义成本函数
def cost_function(params):
    beta, gamma = params[:len(params) // 2], params[len(params) // 2:]
    # qc = QuantumCircuit(num_qubits, num_qubits)
    # qc.h(range(num_qubits))
    B = 1  # 限制1的个数
    dicke_circuit = get_Dicke_state_init_circuit(n, m, B, l)
    qc = QuantumCircuit(num_qubits, num_qubits)
    qc.append(dicke_circuit, range(num_qubits))
    # print(qc)
    length = len(params)
    for i in range(int(length / 2)):
        get_cost_circuit1(gamma[i], qc)
        # get_cost_circuit2(gamma[i], qc)
        get_cost_circuit3(gamma[i], qc)
        get_mixer_circuit(beta[i], qc)
    # 模拟电路
    # 添加测量指令
    qc.measure(range(num_qubits), range(num_qubits))
    backend = Aer.get_backend('qasm_simulator')
    result = execute(qc, backend, seed_simulator=20, shots=10000).result()
    counts = result.get_counts(qc)
    # 计算期望值
    energy = 0
    for bitstring, count in counts.items():
        prob = count / 10000
        bitstring = bitstring[::-1]  # Qiskit 小端序，反转
        x = [int(bitstring[i * m + j]) for i in range(n) for j in range(m)]
        tau = [int(bitstring[n * m + j * l + h]) for j in range(m) for h in range(l)]
        z = [int(bitstring[n * m + m * l + h]) for h in range(l)]

        # 计算 f(x, z, tau)
        c_max = sum(2 ** h * z[h] for h in range(l))
        penalty1 = K1 * sum((sum(x[i * m + j] for j in range(m)) - 1) ** 2 for i in range(n))
        penalty2 = P * sum((sum(x[i * m + j] * p_ij[i][j] for i in range(n)) +
                            sum(2 ** h * (tau[j * l + h] - z[h]) for h in range(l))) ** 2 for j in range(m))
        energy += (c_max + penalty1 + penalty2) * prob
    return energy, counts

In [65]:
# from qiskit_algorithms.optimizers import COBYLA
#
# # 优化 QAOA 参数
# def optimize_qaoa(params):
#     def objective(params):
#         expectation, _ = cost_function(params)
#
#         return expectation
#
#     optimizer = COBYLA(rhobeg = 1.5, tol = 1e-8)
#     # params = np.random.rand(2 * p_max) * 2 * np.pi
#     result = optimizer.minimize(fun=objective, x0=params)
#
#     return result

In [66]:
from scipy.optimize import minimize


def optimize_qaoa(params):
    def objective(params):
        expectation, _ = cost_function(params)

        return expectation

    result = minimize(objective, params, method='Powell',
                      options={'xtol': 1e-5, 'ftol': 1e-5, 'maxiter': 1000})
    return result

In [67]:
# 优化器配置
optimizers = [
    {
        "name": "Powell",
        "func": lambda: {"method": "Powell", "options": {"xtol": 1e-6, "ftol": 1e-6, "maxiter": 1000, "disp": True}}
    }
]

In [68]:
# 插值法初始化函数
def interpolate_parameters(prev_params, p_old, p_new):
    if p_old == 0:
        return np.random.uniform(0, np.pi, p_new), np.random.uniform(0, 2 * np.pi, p_new)
    prev_beta = prev_params[:p_old]
    prev_gamma = prev_params[p_old:]
    new_beta = np.zeros(p_new)
    new_gamma = np.zeros(p_new)
    for i in range(p_new):
        t = i / (p_new - 1) if p_new > 1 else 0
        new_beta[i] = (1 - t) * prev_beta[0] + t * prev_beta[-1]
        new_gamma[i] = (1 - t) * prev_gamma[0] + t * prev_gamma[-1]
    return np.concatenate([new_beta, new_gamma])

In [77]:
# 主程序
import time
import sys

startTime = time.time()
min_energy = float('inf')
init_point = np.array([])
counts = {}
final_result = None

for p1 in range(1, 2):
    min_energy = sys.maxsize
    for k in range(30):
        print(f"第 {p1} 层，第 {k} 个参数")
        init_point_temp = init_point
        # print(f"init_point_temp:{init_point_temp}")
        # beta_p = np.random.uniform(0, np.pi)
        # gamma_p = np.random.uniform(0,  2 * np.pi)
        # if k >0:
        #     init_point_temp = np.delete(init_point_temp, 2 * p1 - 1)
        #     init_point_temp = np.delete(init_point_temp, p1 - 1)
        # init_point_temp = np.insert(init_point_temp, p1 - 1, beta_p)
        # init_point_temp = np.insert(init_point_temp, 2 * p1 - 1, gamma_p)
        if k == 0 and p1 == 1:
            beta_p = np.random.uniform(0, np.pi)
            gamma_p = np.random.uniform(0, 2 * np.pi)
            init_point_temp = np.insert(init_point_temp, p1 - 1, beta_p)
            init_point_temp = np.insert(init_point_temp, 2 * p1 - 1, gamma_p)

            # init_point_temp = np.concatenate([np.random.uniform(0, np.pi, p1),
            #                                  np.random.uniform(0, 2 * np.pi, p1)])
        elif k == 0:
            # 使用插值法从前一层生成新参数
            init_point_temp = interpolate_parameters(init_point, p1 - 1, p1)
        else:
            # 随机调整当前层的参数
            init_point_temp = init_point.copy()
            beta_p = np.random.uniform(0, np.pi)
            gamma_p = np.random.uniform(0, 2 * np.pi)
            init_point_temp = np.delete(init_point_temp, 2 * p1 - 1)
            init_point_temp = np.delete(init_point_temp, p1 - 1)
            init_point_temp = np.insert(init_point_temp, p1 - 1, beta_p)
            init_point_temp = np.insert(init_point_temp, 2 * p1 - 1, gamma_p)
        # print()
        result = optimize_qaoa(init_point_temp)
        optimal_params = result.x
        energy, counts_temp = cost_function(result.x)

        if min_energy > energy:
            min_energy = energy
            init_point = optimal_params
            counts = counts_temp
            final_result = result
            print(f"New best energy: {min_energy}")
            print(f"Optimal parameters: {init_point}")

endTime = time.time()

第 1 层，第 0 个参数


E:\Anaconda3\anaconda\envs\FOOSP\lib\site-packages\scipy\optimize\_optimize.py:3219: RuntimeWarning: overflow encountered in scalar multiply
  t *= temp*temp
E:\Anaconda3\anaconda\envs\FOOSP\lib\site-packages\scipy\optimize\_optimize.py:3221: RuntimeWarning: overflow encountered in scalar multiply
  t -= delta*temp*temp


New best energy: 16472801
Optimal parameters: [0.2654064  2.92792128]
第 1 层，第 1 个参数
第 1 层，第 2 个参数
New best energy: 16333788
Optimal parameters: [3.43858066 5.09121291]
第 1 层，第 3 个参数
New best energy: 15175103
Optimal parameters: [2.66313372 0.82467213]
第 1 层，第 4 个参数
第 1 层，第 5 个参数
第 1 层，第 6 个参数
第 1 层，第 7 个参数
第 1 层，第 8 个参数
第 1 层，第 9 个参数
第 1 层，第 10 个参数
第 1 层，第 11 个参数
第 1 层，第 12 个参数
第 1 层，第 13 个参数
第 1 层，第 14 个参数
第 1 层，第 15 个参数
第 1 层，第 16 个参数
第 1 层，第 17 个参数
第 1 层，第 18 个参数
第 1 层，第 19 个参数
第 1 层，第 20 个参数
第 1 层，第 21 个参数
第 1 层，第 22 个参数
第 1 层，第 23 个参数
第 1 层，第 24 个参数
第 1 层，第 25 个参数
第 1 层，第 26 个参数
第 1 层，第 27 个参数
第 1 层，第 28 个参数
第 1 层，第 29 个参数


In [96]:
import warnings
from qiskit.visualization import plot_histogram

# 忽略弃用警告
warnings.filterwarnings("ignore", category=DeprecationWarning)
# 输出结果
print("优化时间：", endTime - startTime)
print("Optimal parameters:", final_result.x)
print("Optimal value:", final_result.fun)
# 使用最优参数运行量子电路并输出测量结果
final_expectation, final_counts = cost_function(final_result.x)
# print(final_counts)
sorted_dict = {item[0]: item[1] for item in sorted(final_counts.items(), key=lambda item: item[1], reverse=True)}
# 过滤掉计数小于 100 的结果
filtered_dict = {key: value for key, value in sorted_dict.items() if value >= 100}
print("Final expectation value:", final_expectation)
print("Final measurement counts:", filtered_dict)
# plot_histogram(filtered_dict)

优化时间： 506.9285328388214
Optimal parameters: [2.66313372 0.82467213]
Optimal value: 15175103
Final expectation value: 15175103
Final measurement counts: {'1000010001': 205, '1001010010': 199, '1011010010': 197, '1010001010': 188, '1101001001': 188, '1100001001': 187, '1010010001': 186, '1110010001': 185, '1100001010': 180, '1010001001': 180, '1001010001': 179, '1011001010': 178, '1110001001': 177, '1101001010': 177, '1111001010': 175, '1110001010': 174, '1011001001': 173, '1111001001': 173, '1100010001': 173, '1110010010': 173, '1001001001': 172, '1101010010': 172, '1011010001': 171, '1111010010': 170, '1100010010': 168, '1000010010': 166, '1001001010': 166, '1000001010': 165, '1000001001': 161, '1111010001': 161, '1101010001': 160, '1010010010': 142}


In [75]:
# 计算期望值
energy = 0
for bitstring, count in counts.items():
    bitstring = bitstring[::-1]  # Qiskit 小端序，反转
    prob = count / 10000
    x = [int(bitstring[i * m + j]) for i in range(n) for j in range(m)]
    tau = [int(bitstring[n * m + j * l + h]) for j in range(m) for h in range(l)]
    z = [int(bitstring[n * m + m * l + h]) for h in range(l)]

    # 计算 f(x, z, tau)
    c_max = sum(2 ** h * z[h] for h in range(l))
    penalty1 = sum((sum(x[i * m + j] for j in range(m)) - 1) ** 2 for i in range(n))
    penalty2 = sum((sum(x[i * m + j] * p_ij[i][j] for i in range(n)) +
                    sum(2 ** h * (tau[j * l + h] - z[h]) for h in range(l))) ** 2 for j in range(m))
    energy += (c_max + penalty1 + penalty2) * prob
print(energy)

22.468399999999995


In [318]:
expectation = 0
bitstring = "001010"
x = [int(bitstring[i * m + j]) for i in range(n) for j in range(m)]
tau = [int(bitstring[n * m + j * l + h]) for j in range(m) for h in range(l)]
z = [int(bitstring[n * m + m * l + h]) for h in range(l)]

# 计算 f(x, z, tau)
c_max = sum(2 ** h * z[h] for h in range(l))
penalty1 = sum((sum(x[i * m + j] for j in range(m)) - 1) ** 2 for i in range(n))
penalty2 = sum((sum(x[i * m + j] * p_ij[i][j] for i in range(n)) +
                sum(2 ** h * (tau[j * l + h] - z[h]) for h in range(l))) ** 2 for j in range(m))

E = c_max + penalty1 + penalty2
print(c_max)
print(penalty1)
print(penalty2)
# print(e1)
# print(e2)

[[0 0]
 [0 1]
 [1 0]]
[1]
M_i:  0
M_i:  1
M_i:  2
1
0
0
